[Reference](https://medium.com/@rootkibet/prefect-modern-workflow-orchestration-1c5c9fb6f7bd)

In [1]:
pip install prefect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.8/293.8 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 kB 10.1 MB/s eta 0:00:00
  Installing build dependencies ... don

In [3]:
from prefect import flow

@flow
def prefect_flow():
    message = "This is a prefect flow"
    print(message)

if __name__ == "__main__":
    prefect_flow()

In [4]:
!prefect server start

In [5]:
!prefect config set PREFECT_API_URL=http://127.0.0.1:4200/api

In [6]:
from prefect import task, flow

# Define tasks using the @task decorator
@task(name="extract data", log_prints=True, tags="extract")
def extract_data():
    return [1, 2, 3, 4, 5]

@task(name="transform data", log_prints=True, tags="transform")
def transform_data(data):
    return [x * 2 for x in data]

@task(name="load data", log_prints=True, tags="load")
def load_data(transformed_data):
    print("Transformed Data:", transformed_data)

# Create a flow
@flow(name="prefect flow")
def prefect_flow():
    data = extract_data()
    transformed = transform_data(data)
    load_data(transformed)

if __name__ == "__main__":
    prefect_flow()

In [7]:
from prefect.deployments import Deployment
from prefect.server.schemas.schedules import CronSchedule

#import the tutorial script
from tutorial import prefect_flow

# prefect deployment
dep_test = Deployment.build_from_flow(
    flow=prefect_flow,
    name="Prefect flow deployment",
    version=1,
    schedule=(CronSchedule(cron="0 7 * * *", timezone="Africa/Nairobi")),
    work_queue_name="development",
    work_pool_name="default-agent-pool",
)
dep_test.apply()

In [9]:
prefect agent start --pool default-agent-pool --work-queue development